<a href="https://colab.research.google.com/github/shreyasat27/MSc-QT-/blob/main/task_6_matrix_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 0: Symbolic Setup
import sympy as sp
sp.init_printing()

# Define symbolic variables
ε, t_sc, t_sf, g, μB, B = sp.symbols('ε t_sc t_sf g μB B', real=True)

# Define initial Hamiltonian H0 in the basis {|L↑⟩, |L↓⟩, |R↑⟩, |R↓⟩}
H0 = sp.Matrix([
    [ε/2 + g*μB*B/2, t_sc, 0, -t_sf],
    [t_sc, -ε/2 + g*μB*B/2, t_sf, 0],
    [0, t_sf, ε/2 - g*μB*B/2, t_sc],
    [-t_sf, 0, t_sc, -ε/2 - g*μB*B/2]
])

print("🔹 Step 0: Original Hamiltonian H₀")
sp.pprint(H0)

# STEP 1: Rearrangement to {|L↑⟩, |R↓⟩, |L↓⟩, |R↑⟩}
U1 = sp.Matrix([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0],
    [0, 0, 1, 0]
])
H1 = sp.simplify(U1.T * H0 * U1)
print("\n🔹 Step 1: Rearranged Hamiltonian H₁")
sp.pprint(H1)

# STEP 2: Charge mixing rotation — angle φ₀
E0 = sp.sqrt(ε**2 + 4*t_sc**2)
cosφ0 = 2*t_sc / E0
sinφ0 = ε / E0

U2 = sp.Matrix([
    [cosφ0, 0, -sinφ0, 0],
    [0, cosφ0, 0, -sinφ0],
    [sinφ0, 0, cosφ0, 0],
    [0, sinφ0, 0, cosφ0]
])
H2 = sp.simplify(U2.T * H1 * U2)
print("\n🔹 Step 2: After charge mixing rotation H₂")
sp.pprint(H2)

# STEP 3: Spin mixing angles φ₊ and φ₋
Ez = g * μB * B
E_plus = sp.sqrt((E0 + Ez)**2 + 4*t_sf**2)
E_minus = sp.sqrt((E0 - Ez)**2 + 4*t_sf**2)

cosφp = (E0 + Ez) / E_plus
sinφp = 2*t_sf / E_plus
cosφm = (E0 - Ez) / E_minus
sinφm = 2*t_sf / E_minus

U3 = sp.Matrix([
    [cosφp, -sinφp, 0, 0],
    [sinφp, cosφp, 0, 0],
    [0, 0, cosφm, -sinφm],
    [0, 0, sinφm, cosφm]
])
H_final = sp.simplify(U3.T * H2 * U3)
print("\n🔹 Step 3: Final Diagonal Hamiltonian H_final")
sp.pprint(H_final)

# TOTAL UNITARY
U_total = sp.simplify(U1 * U2 * U3)
print("\n✅ Final Total Unitary U_total (as product of all unitaries)")
sp.pprint(U_total)


🔹 Step 0: Original Hamiltonian H₀
⎡B⋅g⋅μB   ε                                        ⎤
⎢────── + ─     t_sc          0           -t_sf    ⎥
⎢  2      2                                        ⎥
⎢                                                  ⎥
⎢            B⋅g⋅μB   ε                            ⎥
⎢   t_sc     ────── - ─      t_sf           0      ⎥
⎢              2      2                            ⎥
⎢                                                  ⎥
⎢                          B⋅g⋅μB   ε              ⎥
⎢    0          t_sf     - ────── + ─      t_sc    ⎥
⎢                            2      2              ⎥
⎢                                                  ⎥
⎢                                        B⋅g⋅μB   ε⎥
⎢  -t_sf         0           t_sc      - ────── - ─⎥
⎣                                          2      2⎦

🔹 Step 1: Rearranged Hamiltonian H₁
⎡B⋅g⋅μB   ε                                        ⎤
⎢────── + ─       0           -t_sf         t_sc   ⎥
⎢  2      2                 

In [2]:
import numpy as np
from scipy.linalg import expm

def construct_total_unitary(epsilon, g, mu_B, B, t_sc, t_sf):
    """
    Construct the total unitary transformation that diagonalizes the Hamiltonian
    through a sequence of 4 unitary operations with mixing angles.

    Returns:
    U_total: The complete unitary transformation
    mixing_angles: Dictionary of all mixing angles (phi0, phi_plus, phi_minus)
    """
    # Calculate all necessary parameters
    E0 = np.sqrt(epsilon**2 + 4*t_sc**2)
    EZ = g * mu_B * B / 2  # Zeeman energy

    # First mixing angle (charge qubit mixing)
    phi0 = np.arctan2(epsilon, 2*t_sc)

    # Second mixing angles (spin mixing)
    E_plus = np.sqrt((E0 + EZ)**2 + 4*t_sf**2)
    phi_plus = np.arctan2(2*t_sf, E0 + EZ)

    E_minus = np.sqrt((E0 - EZ)**2 + 4*t_sf**2)
    phi_minus = np.arctan2(2*t_sf, E0 - EZ)

    # Store all mixing angles
    mixing_angles = {
        'phi0': phi0,
        'phi_plus': phi_plus,
        'phi_minus': phi_minus,
        'E0': E0,
        'E_plus': E_plus,
        'E_minus': E_minus
    }

    # Construct the unitaries step by step

    # U1: Initial state rearrangement
    U1 = np.array([
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]
    ])

    # U2: Charge mixing (phi0 rotation in 1-3 subspace)
    U2 = np.eye(4, dtype=complex)
    U2[0,0] = np.cos(phi0/2)
    U2[0,2] = -np.sin(phi0/2)
    U2[2,0] = np.sin(phi0/2)
    U2[2,2] = np.cos(phi0/2)

    # U3: State rearrangement
    U3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]
    ])

    # U4: Spin mixing (phi_plus and phi_minus rotations)
    U4 = np.eye(4, dtype=complex)

    # Block for phi_plus rotation (1-2 subspace)
    U4[0,0] = np.cos(phi_plus/2)
    U4[0,1] = -np.sin(phi_plus/2)
    U4[1,0] = np.sin(phi_plus/2)
    U4[1,1] = np.cos(phi_plus/2)

    # Block for phi_minus rotation (3-4 subspace)
    U4[2,2] = np.cos(phi_minus/2)
    U4[2,3] = -np.sin(phi_minus/2)
    U4[3,2] = np.sin(phi_minus/2)
    U4[3,3] = np.cos(phi_minus/2)

    # Combine all unitaries (note order of multiplication)
    U_total = U4 @ U3 @ U2 @ U1

    return U_total, mixing_angles

def verify_diagonalization(H, U):
    """Verify that U^† H U is diagonal"""
    H_transformed = U.conj().T @ H @ U
    print("Transformed Hamiltonian (should be diagonal):")
    print(np.real_if_close(H_transformed))
    return H_transformed

# Physical parameters (example values)
epsilon = 1.0          # Energy splitting [meV]
g = 2.0                # g-factor
mu_B = 5.788e-2        # Bohr magneton [meV/T]
B = 0.5                # Magnetic field [T]
t_sc = 0.2             # Spin-conserving tunneling [meV]
t_sf = 0.1             # Spin-flipping tunneling [meV]

# Construct the original Hamiltonian
def build_hamiltonian(eps, g, mu_B, B, t_sc, t_sf):
    H = np.zeros((4,4), dtype=complex)
    EZ = g * mu_B * B / 2
    H[0,0] = eps/2 + EZ
    H[0,1] = t_sc
    H[0,3] = -t_sf
    H[1,0] = t_sc
    H[1,1] = -eps/2 + EZ
    H[1,2] = t_sf
    H[2,1] = t_sf
    H[2,2] = eps/2 - EZ
    H[2,3] = t_sc
    H[3,0] = -t_sf
    H[3,2] = t_sc
    H[3,3] = -eps/2 - EZ
    return H

H = build_hamiltonian(epsilon, g, mu_B, B, t_sc, t_sf)

# Get the total unitary and mixing angles
U_total, angles = construct_total_unitary(epsilon, g, mu_B, B, t_sc, t_sf)

print("Mixing angles (radians):")
for name, value in angles.items():
    if not name.startswith('E'):
        print(f"{name}: {value:.4f} (≈ {np.degrees(value):.1f}°)")

print("\nEnergy parameters:")
print(f"E0: {angles['E0']:.4f} meV")
print(f"E+: {angles['E_plus']:.4f} meV")
print(f"E-: {angles['E_minus']:.4f} meV")

# Verify the diagonalization
H_diag = verify_diagonalization(H, U_total)

# The diagonalized Hamiltonian should have eigenvalues:
expected_eigenvalues = np.array([
    (angles['E_plus'] + angles['E_minus'])/2,
    (angles['E_plus'] - angles['E_minus'])/2,
    (-angles['E_plus'] + angles['E_minus'])/2,
    (-angles['E_plus'] - angles['E_minus'])/2
])

print("\nExpected eigenvalues:", expected_eigenvalues)
print("Diagonal elements:", np.diag(np.real_if_close(H_diag)))

Mixing angles (radians):
phi0: 1.1903 (≈ 68.2°)
phi_plus: 0.1789 (≈ 10.3°)
phi_minus: 0.1886 (≈ 10.8°)

Energy parameters:
E0: 1.0770 meV
E+: 1.1239 meV
E-: 1.0670 meV
Transformed Hamiltonian (should be diagonal):
[[ 0.11353137 -0.55435394  0.0890223   0.05719646]
 [-0.55435394 -0.1145477  -0.0608539   0.08534198]
 [ 0.0890223  -0.0608539  -0.49866977  0.09999884]
 [ 0.05719646  0.08534198  0.09999884  0.49968609]]

Expected eigenvalues: [ 1.09545787  0.02845325 -0.02845325 -1.09545787]
Diagonal elements: [ 0.11353137 -0.1145477  -0.49866977  0.49968609]
